# Interview Notes: LangChain Agent Types for GenAI Roles

## Overview of Agents
Agents are autonomous systems that use LLMs to determine which actions to take and in what order. They maintain a reasoning loop: observe → think → act → repeat until task completion.

**Core Components:**
- **Agent:** The decision-making LLM that chooses actions
- **Tools:** Functions the agent can call (APIs, databases, search engines)
- **AgentExecutor:** Orchestrates the agent-tool interaction loop
- **Memory:** Stores conversation history and context

---

## 1. ZeroShotAgent

### What It Is
A versatile agent that selects tools based on their descriptions alone, without examples. It dynamically decides which tool to use for each step based on the current task.

### Key Characteristics
- Uses the **ReAct (Reasoning + Acting)** framework
- Works with any tool, as long as it has a clear description
- No fine-tuning or few-shot examples required
- Single-turn action selection per iteration

### Prompt Structure
```
Answer the following questions as best you can. You have access to the following tools:

{tool_descriptions}

Use this format:
Question: the input question
Thought: reasoning about what to do
Action: the tool to use
Action Input: the input to the tool
Observation: the result from the tool
... (repeat Thought/Action/Observation as needed)
Thought: I now know the final answer
Final Answer: the final response
```

### Use Cases
- Quick prototyping with new tools
- Tasks requiring flexible tool selection
- General-purpose question answering with tools

### Interview Tips
- Explain the ReAct framework: interleaving reasoning and actions
- Mention limitations: can struggle with complex multi-step reasoning
- Discuss when to use vs. other agent types

---

## 2. ConversationalAgent

### What It Is
An agent optimized for multi-turn conversations that maintains context across interactions while using tools.

### Key Characteristics
- Built-in **conversation memory** (ConversationBufferMemory)
- Maintains chat history automatically
- More natural dialogue flow
- Can reference previous exchanges

### Prompt Structure
```
Assistant is a conversational agent designed to help with tasks using tools.

TOOLS:
{tool_descriptions}

Previous conversation:
{chat_history}

Current conversation:
Human: {input}
{agent_scratchpad}
```

### Differences from ZeroShotAgent
- Explicitly includes chat history in prompts
- Better at maintaining context over multiple turns
- Ideal for chatbots and interactive applications

### Use Cases
- Customer service chatbots with tool access
- Interactive data analysis assistants
- Multi-turn task completion (e.g., booking systems)

### Interview Tips
- Emphasize memory management strategies (buffer, summary, knowledge graph)
- Discuss token limits and conversation pruning
- Explain how to handle context window constraints

---

## 3. StructuredChatAgent

### What It Is
An agent that can handle **complex tool inputs** requiring structured data (JSON objects, nested parameters).

### Key Characteristics
- Supports multi-parameter tools
- Uses JSON for action inputs
- Better handling of complex tool schemas
- Can pass dictionaries, lists, nested structures

### Example Tool Input
```json
{
  "action": "database_query",
  "action_input": {
    "query": "SELECT * FROM users WHERE age > 25",
    "database": "production",
    "timeout": 30,
    "return_format": "json"
  }
}
```

### When to Use
- Tools requiring multiple parameters
- APIs with complex request structures
- Database queries with multiple filters
- Tools needing nested JSON inputs

### Advantages Over Basic Agents
- More expressive tool calls
- Reduces parsing errors with structured inputs
- Better for enterprise integrations

### Interview Tips
- Contrast with ZeroShotAgent's simple string inputs
- Discuss JSON schema validation for tools
- Mention error handling for malformed inputs

---

## 4. ReAct Agent

### What It Is
The foundational agent implementing the **ReAct (Reasoning + Acting) paradigm** from the DeepMind paper.

### Core Concept
Combines verbal reasoning traces with actions:
1. **Thought:** Internal reasoning step
2. **Action:** External action via tool
3. **Observation:** Result from action
4. Repeat until final answer

### ReAct Framework Benefits
- **Interpretability:** Reasoning is visible
- **Debuggability:** Can trace decision path
- **Error recovery:** Can self-correct based on observations
- **Synergy:** Reasoning improves actions, actions ground reasoning

### Example Trace
```
Question: What's the weather in Paris and should I bring an umbrella?

Thought: I need current weather data for Paris
Action: weather_api
Action Input: Paris, France
Observation: Temperature 18°C, rain expected this afternoon

Thought: Rain is forecasted, so an umbrella would be useful
Final Answer: The weather in Paris is 18°C with rain expected this afternoon. Yes, you should bring an umbrella.
```

### Interview Tips
- Cite the original ReAct paper (Yao et al., 2022)
- Explain how it differs from pure chain-of-thought reasoning
- Discuss limitations: verbose prompts, higher token usage

---

## 5. OpenAI Tools Agent (Function Calling Agent)

### What It Is
A modern agent leveraging **OpenAI's native function calling** API (also supported by other providers like Anthropic).

### Key Characteristics
- Uses model's built-in function calling capabilities
- More reliable tool selection
- Structured outputs guaranteed
- Lower latency than prompt-based agents
- Native support in GPT-3.5-turbo, GPT-4, Claude 3+

### How It Works
```python
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
                },
                "required": ["location"]
            }
        }
    }
]
```

### Advantages
- **Reliability:** Model explicitly trained for function calling
- **Structured outputs:** JSON schema validation
- **Parallel tool calls:** Can invoke multiple tools simultaneously
- **Lower prompt tokens:** No verbose ReAct prompting needed

### Use Cases
- Production systems requiring reliability
- Applications with strict output formatting
- Multi-tool orchestration
- Real-time applications (lower latency)

### Interview Tips
- Explain function calling vs. prompt-based tool use
- Discuss parallel function calling for efficiency
- Mention model support (OpenAI, Anthropic, Google, etc.)
- Compare prompt injection resistance vs. traditional agents

---

## Comparison Table

| Agent Type | Best For | Tool Input | Memory | Complexity |
|------------|----------|------------|---------|------------|
| ZeroShotAgent | Quick prototyping, simple tools | String | Optional | Low |
| ConversationalAgent | Multi-turn chats | String | Built-in | Medium |
| StructuredChatAgent | Complex API calls | JSON | Optional | Medium |
| ReAct Agent | Transparent reasoning | String | Optional | Medium |
| OpenAI Tools Agent | Production reliability | JSON Schema | Optional | Low-Medium |

---

## Advanced Interview Topics

### Agent Execution Strategies
- **Max iterations:** Prevent infinite loops
- **Early stopping:** Handle tool failures
- **Timeout handling:** Manage long-running tools

### Error Handling Patterns
```python
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    max_iterations=5,
    handle_parsing_errors=True,
    return_intermediate_steps=True
)
```

### Custom Agent Creation
- Extending base Agent class
- Custom prompt templates
- Tool result parsing
- Output formatting

### Production Considerations
- **Token optimization:** Minimize prompt size
- **Caching:** Reduce repeated LLM calls
- **Monitoring:** Track tool usage and failures
- **Fallback strategies:** Handle model unavailability
- **Cost management:** Balance capability vs. expense

### Multi-Agent Systems
- Agent collaboration patterns
- Hierarchical agent structures
- Specialized vs. generalist agents
- Communication protocols between agents

---

## Common Interview Questions

**Q: When would you choose StructuredChatAgent over OpenAI Tools Agent?**
A: StructuredChatAgent when using models without native function calling or when you need more control over the prompting strategy. OpenAI Tools Agent for production reliability with supported models.

**Q: How do you prevent agent hallucinations with tools?**
A: Clear tool descriptions, output validation, structured outputs, setting max iterations, and using models with function calling support.

**Q: How would you optimize agent performance in production?**
A: Caching tool results, minimizing prompt tokens, using parallel tool calls, implementing timeouts, monitoring tool reliability, and choosing appropriate models for cost-performance trade-offs.

**Q: What's the difference between ReAct and Chain-of-Thought?**
A: Chain-of-Thought is pure reasoning without actions. ReAct interleaves reasoning with real-world actions via tools, allowing the agent to ground its thinking in observations.

In [3]:
"""
ZeroShotAgent (LangChain 1.x equivalent)
---------------------------------------
• In "classic" LangChain, ZeroShotAgent was commonly used via initialize_agent(..., AgentType.ZERO_SHOT_*).
• In LangChain 1.x, the recommended approach is create_agent(...), which runs the core loop:
  model -> (optional tool calls) -> final answer.
• Use this for single-turn tasks where you want the LLM to pick tools based on tool docstrings.

ZeroShotAgent
-------------
• Uses LLM reasoning only (no memory of prior conversation)
• Chooses tools based on tool descriptions
• Best for single-turn reasoning + tool usage
• No structured output enforcement

"""

from langchain.agents import create_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the product."""
    return a * b

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_agent(
    model=model,
    tools=[multiply],
    system_prompt="You are a helpful assistant. Use tools when needed."
)

result = agent.invoke({
    "messages": [
        {"role": "user", "content": "What is 7 multiplied by 8?"}
    ]
})

print(result["messages"][-1].content)


7 multiplied by 8 is 56.


In [5]:
"""
ConversationalAgent (LangChain 1.x equivalent)
----------------------------------------------
• "ConversationalAgent" in classic LangChain focused on multi-turn chat with memory.
• In LangChain 1.x, conversation history is carried in the agent state via the 'messages' list.
• You keep/append messages between turns and pass them back in on the next invoke().
• Optional: add SummarizationMiddleware when message history grows large (not shown here).

ConversationalAgent
-------------------
• Designed for multi-turn chat
• Maintains conversation context
• Still uses tools when required
• Ideal for chatbots + assistants
"""

from langchain.agents import create_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI

@tool
def get_weather(city: str) -> str:
    """Return a simple weather string for a given city."""
    return f"Weather in {city}: Sunny"

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_agent(
    model=model,
    tools=[get_weather],
    system_prompt="You are a conversational assistant. Keep context from prior messages."
)

# --- Turn 1
state = {
    "messages": [
        {"role": "user", "content": "What is the weather in Bengaluru?"}
    ]
}
result1 = agent.invoke(state)

assistant_msg_1 = result1["messages"][-1]
print("Assistant:", assistant_msg_1.content)

# --- Turn 2 (carry forward messages)
state["messages"].append({"role": "assistant", "content": assistant_msg_1.content})
state["messages"].append({"role": "user", "content": "And what about Mysuru?"})

result2 = agent.invoke(state)
print("Assistant:", result2["messages"][-1].content)


Assistant: The weather in Bengaluru is sunny.
Assistant: The weather in Mysuru is also sunny.


In [9]:
"""
StructuredChatAgent
-------------------
• Forces structured (JSON-like) responses
• Best for APIs, backend systems
• Prevents free-text hallucination
• Excellent for automation pipelines

StructuredChatAgent (LangChain 1.x equivalent)
----------------------------------------------
• Classic StructuredChatAgent tried to enforce structured / JSON-like outputs.
• In LangChain 1.x, the built-in way is response_format=ToolStrategy(Schema) or ProviderStrategy(Schema).
• ToolStrategy works with tool-calling models and returns structured data in result["structured_response"].
"""

from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain_openai import ChatOpenAI

class TicketRequest(BaseModel):
    destination: str
    date: str
    num_passengers: int

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_agent(
    model=model,
    tools=[],  # tools optional; structured output can be used even without tools
    response_format=ToolStrategy(TicketRequest),
    system_prompt="Extract the booking request as structured data."
)

result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Book 2 tickets to Delhi for 2026-02-10"}
    ]
})

print("Structured:", result["structured_response"])   # -> TicketRequest(...)


Structured: destination='Delhi' date='2026-02-10' num_passengers=2


In [11]:
"""
ReAct Agent
-----------
• Explicit Reasoning → Action → Observation loop
• Thinks step-by-step before using tools
• Most important agent conceptually
• Foundation of modern Agentic AI

ReAct Agent (LangChain 1.x equivalent)
--------------------------------------
• ReAct = Reasoning + Acting via tools (loop until no more tool calls).
• In LangChain 1.x, create_agent(...) is the standard agent loop that supports tool calling naturally.
• If you stream, you can see intermediate steps/messages (tool calls, tool results, etc.).
"""

from langchain.agents import create_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI

@tool
def search(query: str) -> str:
    """Search the web (mock). Return quick 'results' for a query."""
    return f"[MOCK] Top results for: {query}"

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_agent(
    model=model,
    tools=[search],
    system_prompt="You are a research assistant. Use the search tool when you need facts."
)

result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Find what ReAct means in agentic AI and summarize in 2 lines."}
    ]
})

print(result["messages"][-1].content)


The ReAct framework in agentic AI refers to a method that combines reasoning and acting, enabling AI systems to make decisions based on both logical reasoning and real-time interactions with their environment. This approach enhances the adaptability and effectiveness of AI agents in dynamic situations.


In [13]:
"""
OpenAI Tools Agent
------------------
• Uses OpenAI function calling / tools API
• Most reliable & production-ready
• Clean tool invocation (no text parsing)
• Recommended agent going forward

OpenAI Tools Agent (LangChain 1.x equivalent)
---------------------------------------------
• In classic LangChain, "OpenAI Tools Agent" referred to OpenAI function/tool calling based agents.
• In LangChain 1.x, create_agent(...) + a tool-calling model (e.g., ChatOpenAI) gives you that behavior.
• This is the most "modern" default agent style in v1: clean tool calls, reliable execution loop.
"""

from langchain.agents import create_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI

@tool
def add(a: int, b: int) -> int:
    """Add two integers and return the sum."""
    return a + b

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_agent(
    model=model,
    tools=[add],
    system_prompt="Use tools for math instead of guessing."
)

result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Add 15 and 27"}
    ]
})

print(result["messages"][-1].content)


The sum of 15 and 27 is 42.
